In [27]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
import json

# Import API key
from api_keys import api_key

# Incorporated citipy to determine city based on latitude and longitude
from citipy import citipy

# Output File (CSV)
output_data_file = "output_data/cities.csv"

# Range of latitudes and longitudes
lat_range = (-90, 90)
lng_range = (-180, 180)

In [28]:
# List for holding lat_lngs and cities
lat_lngs = []
cities = []

# Create a set of random lat and lng combinations
lats = np.random.uniform(low=-90.000, high=90.000, size=1500)
lngs = np.random.uniform(low=-180.000, high=180.000, size=1500)
lat_lngs = zip(lats, lngs)

# Identify nearest city for each lat, lng combination
for lat_lng in lat_lngs:
    city = citipy.nearest_city(lat_lng[0], lat_lng[1]).city_name
    
    # If the city is unique, then add it to a our cities list
    if city not in cities:
        cities.append(city)

# Print the city count to confirm sufficient count
len(cities)

610

In [29]:
## API call: 
## api.openweathermap.org/data/2.5/weather?q={city name}
## api.openweathermap.org/data/2.5/weather?q={city name},{country code}


In [80]:
print("Beginning Data Retrieval")
print("-----------------------------")
cities_df = []
record_count=1
set_count=1
for x in cities[0:3]:
    response = requests.get("http://api.openweathermap.org/data/2.5/weather?", params=dict(q=x, units="imperial", APPID=api_key))
    if response.status_code != 200:
        print("City not found. Skipping...")
    else:
        response = requests.get("http://api.openweathermap.org/data/2.5/weather?", params=dict(q=x, units="imperial", APPID=api_key)).json()
        print(f"Processing Record {record_count} of Set {set_count} | {x}")
        cities_df.append(response)
        record_count += 1
        if record_count == 51:
            record_count = 1
            set_count += 1
            
print("-----------------------------")
print("Data Retrieval Complete")
print("-----------------------------")

Beginning Data Retrieval
-----------------------------
City not found. Skipping...
Processing Record 1 of Set 1 | oranjemund
Processing Record 2 of Set 1 | georgetown
-----------------------------
Data Retrieval Complete
-----------------------------


In [125]:
DF = pd.DataFrame(cities_df)
DF

,base,clouds,cod,coord,dt,id,main,name,sys,visibility,weather,wind
0,stations,{'all': 68},200,"{'lon': 16.43, 'lat': -28.55}",1545184628,3354071,"{'temp': 61.57, 'pressure': 1028.28, 'humidity...",Oranjemund,"{'message': 0.0032, 'country': 'ZA', 'sunrise'...",NaN,"[{'id': 803, 'main': 'Clouds', 'description': ...","{'speed': 0.58, 'deg': 353.501}"
1,stations,{'all': 75},200,"{'lon': -58.16, 'lat': 6.8}",1545181200,3378644,"{'temp': 77.68, 'pressure': 1013, 'humidity': ...",Georgetown,"{'type': 1, 'id': 8776, 'message': 0.0037, 'co...",10000.0,"[{'id': 803, 'main': 'Clouds', 'description': ...","{'speed': 11.41, 'deg': 50}"


In [81]:
test = json.dumps(cities_df, indent=4, sort_keys=True)

In [86]:
test2 = json.loads(test)

{'base': 'stations',
 'clouds': {'all': 68},
 'cod': 200,
 'coord': {'lat': -28.55, 'lon': 16.43},
 'dt': 1545184628,
 'id': 3354071,
 'main': {'grnd_level': 1028.28,
  'humidity': 95,
  'pressure': 1028.28,
  'sea_level': 1029,
  'temp': 61.57,
  'temp_max': 61.57,
  'temp_min': 61.57},
 'name': 'Oranjemund',
 'sys': {'country': 'ZA',
  'message': 0.0032,
  'sunrise': 1545191548,
  'sunset': 1545241818},
 'weather': [{'description': 'broken clouds',
   'icon': '04n',
   'id': 803,
   'main': 'Clouds'}],
 'wind': {'deg': 353.501, 'speed': 0.58}}

In [90]:
## list of dictionaries, list of oe dictionary would just come back as dict
type(test2)

list

In [91]:
new_dict = {item['name']:item for item in test2}


In [92]:
type(new_dict)

dict

In [111]:
test_df= pd.DataFrame.from_dict(new_dict, orient="index",columns=['clouds','main'])

# train = pd.DataFrame.from_dict(dict_train, orient='index')
# train.reset_index(level=0, inplace=True)

In [105]:
##ttemperatures in sub dict MAIN
test_df["clouds"]["all"]

,clouds,main
Georgetown,{'all': 75},"{'humidity': 78, 'pressure': 1013, 'temp': 77...."
Oranjemund,{'all': 68},"{'grnd_level': 1028.28, 'humidity': 95, 'press..."
